In [130]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


save_steps = False
low_threshold = 0
iteration_num = 5
src = cv2.imread('img.png')

In [131]:
def update():
	img = src.copy()
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	_, thresh = cv2.threshold(gray, low_threshold, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
	kernel = np.ones((3, 3), np.uint8)
	sure_bg = cv2.dilate(thresh, kernel, iterations=iteration_num)
	dist_transform = cv2.distanceTransform(sure_bg, 1, 5)
	dist_transform = np.uint8(dist_transform)
	_, sure_fg = cv2.threshold(dist_transform, low_threshold, 255, 0)

	sure_fg = np.uint8(sure_fg)
	unknown = cv2.subtract(sure_bg, sure_fg)

	_, markers = cv2.connectedComponents(sure_fg)
	markers = markers + 1

	markers[unknown == 255] = 0

	markers = cv2.watershed(img, markers)

	img[markers == -1] = (0, 0, 255)
	# img = cv2.resize(img, (800, 800))
	cv2.imshow("img", img)

	if save_steps:
		plt.rcParams['figure.dpi'] = 250
		plt.subplot(241)
		plt.imshow(cv2.cvtColor(src, cv2.COLOR_BGR2RGB)),
		plt.title('Original')
		plt.axis('off')
		plt.subplot(242)
		plt.imshow(thresh, cmap='gray'),
		plt.title('Threshold')
		plt.axis('off')
		plt.subplot(243)
		plt.imshow(sure_bg, cmap='gray'),
		plt.title('Dilate')
		plt.axis('off')
		plt.subplot(244)
		plt.imshow(dist_transform, cmap='gray'),
		plt.title('Dist Transform')
		plt.axis('off')
		plt.subplot(245)
		plt.imshow(sure_fg, cmap='gray'),
		plt.title('Threshold')
		plt.axis('off')
		plt.subplot(246)
		plt.imshow(unknown, cmap='gray'),
		plt.title('Unknow')
		plt.axis('off')
		plt.subplot(247)
		plt.imshow(np.abs(markers), cmap='jet'),
		plt.title('Markers')
		plt.axis('off')
		plt.subplot(248)
		plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)),
		plt.title('Result')
		plt.axis('off')
		plt.savefig('steps.png')


def change_low_threshold(val):
	global low_threshold
	low_threshold = val
	update()


def change_iteration_num(val):
	global iteration_num
	iteration_num = val
	update()

In [132]:
cv2.namedWindow('Control')
cv2.createTrackbar('thresh low', 'Control', 20, 255, change_low_threshold)
cv2.createTrackbar('iterations', 'Control', 5, 100, change_iteration_num)
cv2.waitKey(0)
cv2.destroyAllWindows()